# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
#библиотеки
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error


In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.info()
data.info()
#пропусков нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.head()


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"></a></h2>

<b>Все отлично!👍:  </b> С данными познакомились, проведен первичный аналитический осмотр!) Полученная информация поможет нам в дальнейшем с предобработкой данных. 
        
- Хочу тут затронуть термин мультиколлениарности, чтобы было представление о нем. В наших данных есть мультиколлинеарность в фичах calls и minutes. Желательно проверить это методом `.corr(method='spearman')`       
        
    В нашем проекте мы сосредоточены на предсказаниях, поэтому проблема мультиколлинеарности, если она и есть, для нас не страшна, на предсказаниях она не сказывается (она влияет на значения коэффициентов логистической регрессии `.coef_` (это, кстати, <a href="https://betterdatascience.com/feature-importance-python/">о важности признаков</a>), для скоррелированных признаков они смещены, поэтому их интерпретация становится невозможной, но для нас сейчас это не имеет значения). 
- Рекомендую также тебе посмотреть метод `sns.pairplot` с параметром `hue`, в который мы передаем целевой признак.
</div>
        

## Разбейте данные на выборки

In [6]:
#признаки: calls, minutes, messages, mb_used
#целевой признак: is_ultra
features = data.drop('is_ultra', axis=1)
target = data['is_ultra']

In [7]:
#делим в соотношении 3:1:1
#старое

#features_train, features_others, target_train, target_others = train_test_split(
 #   features, target, test_size=0.2, random_state=12345)
#features_test, features_valid, target_test, target_valid = train_test_split(
  #  features, target, test_size=0.2, random_state=12345)


In [8]:
data_train, data_other =train_test_split(
    data, test_size=0.4, random_state=12345) 
data_test, data_valid = train_test_split(
    data_other, test_size=0.5, random_state=12345)

In [9]:
print(data_train.shape[0])
print(data_test.shape[0])
print(data_valid.shape[0])

1928
643
643


In [10]:
#или
features_train, features_others, target_train, target_others = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_others, target_others, test_size=0.5, random_state=12345)

In [11]:
print(features_train.shape[0])
print(features_test.shape[0])
print(features_valid.shape[0])

1928
643
643


<br/>
<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"></h2>
    
<b>На доработку🤔:</b> Восстановил его
    
- Сейчас у тебя выборки пересекаются, т.к. они созданы из одной выборки df.        
Исправь, пожалуйста.
- Датасет следует разбить так, чтобы доля обучающей выборки была от 50% до 80%, а доля тестовой и валидационной выборки: от 10% до 25%. Одновременно, валидационная и тестовая выборки не должны отличаться по размеру в два раза или более. я бы рекомендовал придерживаться соотношения 60%-20%-20% (обучающая, валидационная, тестовая). Исправь, пожалуйста.    
 </div>

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Исправлено

## Исследуйте модели

### Решающее дерево

In [12]:
#находим accurracy
#старое

#best_model = None
#best_result = 0
#for depth in range(1, 11):
 #   model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
  #  model.fit(features,target) # обучите модель
  #  predictions = model.predict(features) # получите предсказания модели
  #  result = accuracy_score(target, predictions) # посчитайте качество модели
  #  if result > best_result:
   #     best_model = model
   #     best_result = result       
#print("Accuracy лучшей модели:", best_result)

In [13]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)


In [14]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучаем модель с заданной глубиной дерева
    model.fit(features_train,target_train) # обучаем модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) # получаем предсказания модели на валидационной
    result = accuracy_score(target_valid, predictions_valid) # считаем качество модели
    if result > best_result:
        best_model = model
        best_result = result       
print("Accuracy лучшей модели:", best_result)

Accuracy лучшей модели: 0.7993779160186625


<br/>
<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера  <a class="tocSkip"></h2>

    
<b>На доработку🤔:</b>
    
   
- На тренировочной выборке мы обучаем модель, а на валидационной подбираем лучшие гиперпараметы 
    
- Тестовую выборку мы используем как «слепую» выборку для определения качества на неизвестных ранее данных для ОДНОЙ лучшей модели - победителя (у нее самое высокое значение метрики accuracy на валидационной выборке). Тестовую выборку мы используем в самом конце в разделе «4. Проверьте качество модели на тестовой выборке.» 


<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Вижу, хорошо

In [15]:
#находим accuracy по валидационной выборке
data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345)
for depth in range(1, 20):
    model =  DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit (features_train, target_train)
   

    predictions_valid =model.predict(features_valid) 

    print("max_depth =", depth, ": ", end='') 
    print(accuracy_score(target_valid, predictions_valid)) 

max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255
max_depth = 6 : 0.776049766718507
max_depth = 7 : 0.7993779160186625
max_depth = 8 : 0.7931570762052877
max_depth = 9 : 0.7807153965785381
max_depth = 10 : 0.7884914463452566
max_depth = 11 : 0.7744945567651633
max_depth = 12 : 0.7807153965785381
max_depth = 13 : 0.7713841368584758
max_depth = 14 : 0.76049766718507
max_depth = 15 : 0.7713841368584758
max_depth = 16 : 0.749611197511664
max_depth = 17 : 0.7480559875583204
max_depth = 18 : 0.749611197511664
max_depth = 19 : 0.7433903576982893


 7 : 0.7993779160186625

### Случайный лес

In [16]:
#старое

#best_model = None
#best_result = 0
#for est in range(1, 50):
 #   model = RandomForestClassifier(random_state=12345, n_estimators=est) 
   
  #  model.fit(features,target)
  #  predictions = model.predict(features) 
   # result = accuracy_score(target, predictions) 
   # if result > best_result:
   #     best_model = model
    #    best_result = result       
#print("Accuracy лучшей модели:", best_result)

In [17]:
best_model = None
best_result = 0
for est in range(10, 100):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
   
    model.fit (features_train, target_train)
   

    predictions_valid =model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions_valid) 
    if result > best_result:
        best_model = model
        best_result = result       
print("Accuracy лучшей модели:", best_result)

Accuracy лучшей модели: 0.7962674961119751


In [18]:
#какой из вариантов будет более корректен? Номер 1




#for depth in range(10, 100):
  #  model =  RandomForestClassifier(random_state=12345, max_depth=depth) 
  #  model.fit (features_train, target_train)
   
  #  predictions_valid = model.predict(features_valid) 

   # print("max_depth =", depth, ": ", end='') 
   # print(accuracy_score(target_valid, predictions_valid)) 
    
  

In [19]:
#номер 2


best_est = 0
best_accuracy = 0

for est in range(10, 200, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    print('Деревья', est,'Accuracy',accuracy)
    if accuracy > best_accuracy:
        best_est = est
        best_accuracy = accuracy

print('Лучшее', best_est,'Accuracy', best_accuracy)  

Деревья 10 Accuracy 0.7807153965785381
Деревья 20 Accuracy 0.7822706065318819
Деревья 30 Accuracy 0.7853810264385692
Деревья 40 Accuracy 0.7931570762052877
Деревья 50 Accuracy 0.7931570762052877
Деревья 60 Accuracy 0.7947122861586314
Деревья 70 Accuracy 0.7931570762052877
Деревья 80 Accuracy 0.7962674961119751
Деревья 90 Accuracy 0.7916018662519441
Деревья 100 Accuracy 0.7853810264385692
Деревья 110 Accuracy 0.7900466562986003
Деревья 120 Accuracy 0.7853810264385692
Деревья 130 Accuracy 0.7869362363919129
Деревья 140 Accuracy 0.7869362363919129
Деревья 150 Accuracy 0.7884914463452566
Деревья 160 Accuracy 0.7931570762052877
Деревья 170 Accuracy 0.7869362363919129
Деревья 180 Accuracy 0.7884914463452566
Деревья 190 Accuracy 0.7869362363919129
Лучшее 80 Accuracy 0.7962674961119751


80 Accuracy 0.7962674961119751

<div class="alert alert-warning">
    <h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b>
    

Лес из одного дерева - странная штука) Желательно выбирать количество деревьев побольше и больший шаг при переборе.
        
Советую сделать разбег в 10 деревьев, начинать с 10 штук, максимум 200 - 500. 
       
 </div>

<div class="alert alert-warning">
    <h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 
        
Оба варианта не совсем правильные:
- не стоит перебирать глубину дерева max_depth больше, чем 10, как правило начинается переобучение        
        
- для числа деревьев n_estimators можно было задать range (10, 200, 10).  
-  строка `data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345)` тут излишняя и ее стоит удалить       

### Логистическая регрессия

In [20]:

best_model = None
best_result = 0
for est in range(1, 100):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
   
    model.fit(features_train,target_train) 
    
    predictions_valid = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions_valid) 
    if result > best_result:
        best_model = model
        best_result = result       
print("Accuracy лучшей модели:", best_result)


Accuracy лучшей модели: 0.6842923794712286


In [21]:
data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345)



for depth in range(1, 20):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
    model.fit (features_train, target_train)
   

    predictions_valid = model.predict(features_valid) 

    print("max_depth =", depth, ": ", end='') 
    print(accuracy_score(target_valid, predictions_valid)) 

max_depth = 1 : 0.6842923794712286
max_depth = 2 : 0.6842923794712286
max_depth = 3 : 0.6842923794712286
max_depth = 4 : 0.6842923794712286
max_depth = 5 : 0.6842923794712286
max_depth = 6 : 0.6842923794712286
max_depth = 7 : 0.6842923794712286
max_depth = 8 : 0.6842923794712286
max_depth = 9 : 0.6842923794712286
max_depth = 10 : 0.6842923794712286
max_depth = 11 : 0.6842923794712286
max_depth = 12 : 0.6842923794712286
max_depth = 13 : 0.6842923794712286
max_depth = 14 : 0.6842923794712286
max_depth = 15 : 0.6842923794712286
max_depth = 16 : 0.6842923794712286
max_depth = 17 : 0.6842923794712286
max_depth = 18 : 0.6842923794712286
max_depth = 19 : 0.6842923794712286


**Вывод**

В ходе исследования были получены следующие данные:

Accuracy лучшей модели решающего дерева:  0.7993779160186625

Accuracy лучшей модели случайного леса: 0.7962674961119751

Accuracy лучшей модели логистической регрессии: 0.6842923794712286

Лучший результат показывает решающее дерево. Не сильно отстает случайный лес. Хуже всего себя показал метод логистической регрессии.



<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"></a></h2>

<b>Все отлично!👍:</b>

В остальном, имеются корректные шаги в исследовании моделей:
    
 - исследовано более 5 значений гиперпараметра

 - перебор гиперпараметров осуществляется в цикле 
    
</div>

<br/>
<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>
    
<b>На доработку🤔:</b>
    
Тут следует еще указать, какие модели работают лучше, а какие — хуже.
Добавь, пожалуйста.
 </div>

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Вижу, хорошо

## Проверьте модель на тестовой выборке

In [22]:
#Решающее дерево
# старое
#best_model = None
#best_result = 0
#for depth in range(1, 11):
   # model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
  #  model.fit(features,target) 
   # predictions_test = model.predict(features_test) 
   # result = accuracy_score(target_test, predictions_test) 
   # if result > best_result:
    #    best_model = model
    #    best_result = result       
#print("Accuracy лучшей модели:", best_result)

In [23]:
#исправленная
model_wood = DecisionTreeClassifier(random_state=12345, max_depth=7) # обучили модель по дереву с глубиной максимальной acc (7)

model_wood.fit(features_train,target_train) # обучили на обучающей
predictions = model_wood.predict(features_test) #предсказали на тестовой
accuracy = accuracy_score(predictions, target_test) # считаем на предсказании + тестовой 
print('Точность модели на тестовой выборке:', accuracy) 

Точность модели на тестовой выборке: 0.7822706065318819


<br/>
<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>

    
<b>На доработку🤔:</b>
    
Некорректно как-либо использовать тестовую выборку в подборе гиперпараметров или в выборе модели, а также обучать модель на тестовых данных. Нам следует обучить модель на обучающей выборке и  сделать предсказание методом `predict` на тестовой выборке для ОДНОЙ лучшей модели - победителя с предыдущего шага и рассчитать итоговую метрику. Исправь, пожалуйста.
</div>

<br/>
<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"></h2>

    
<b>На доработку🤔:</b>
    
Некорректно как-либо использовать тестовую выборку в подборе гиперпараметров, а также обучать модель на полном датасете. Нам следует обучить модель на обучающей выборке и  сделать предсказание методом `predict` на тестовой выборке для ОДНОЙ лучшей модели - победителя с предыдущего шага и рассчитать итоговую метрику. Исправь, пожалуйста.
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера v.3 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b>  Теперь все четко)

In [24]:
#случайный лес
#best_model = None
#best_result = 0
#for est in range(10, 100):
  #  model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
   
  #  model.fit(features_test,target_test) 
# обучите модель
  #  predictions_test = model.predict(features_test) # получите предсказания модели
  #  result = accuracy_score(target_test, predictions_test) # посчитайте качество модели
  #  if result > best_result:
     #   best_model = model
    #    best_result = result       
#print("Accuracy лучшей модели:", best_result)

In [25]:

# Логистичесская регрессия

#best_model = None
#best_result = 0
#for est in range(1, 11):
 #   model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) # обучите модель с заданным количеством деревьев
   
  #  model.fit(features_test,target_test) # обучите модель
 #   predictions_test = model.predict(features_test) # получите предсказания модели
  #  result = accuracy_score(target_test, predictions_test) # посчитайте качество модели
 #   if result > best_result:
 #       best_model = model
 #      best_result = result       
#print("Accuracy лучшей модели:", best_result)

**Вывод**

Accuracy лучшей модели решающего дерева: 0.7822706065318819



<div class="alert alert-warning">
    <h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b>
    
Тестирование положено проводить для одной лучшей модели. На предыдущем шаге мы должны были настроить модели и выбрать одну наилучшую, опираясь на метрики данные заказчиком (это может быть качество, время обучения, скорость предсказания и т.д.). Тестированием мы моделируем работу модели на новых незнакомых ей данных, которые ни разу не использовались ни при тренировке, ни при валидации, и проверяем, не словили ли мы переобучение. А эти данные могут быть смещенными, с выбросами и т.д. То есть по таким данным некорректно заново переопределять модель-победитель.  Советую тебе статью, рассматривающую разные способы валидации моделей машинного обучения: https://towardsdatascience.com/validating-your-machine-learning-model-25b4c8643fb7
        
Метрики качества на тестовой выборке для всех моделей можно вычислить только с целью исследования их смещения относительно аналогичных метрик на валидационной выборке. Но это не означает, что модель-победитель должна выбираться исходя из сравнения метрик, полученных на тестовой выборке.        

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Вижу, хорошо

## (бонус) Проверьте модели на адекватность

Не понимаю, что нужно сделать

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> 
    
Этап тестирования рекомендуется дополнять введением в работу baseline модели. Сравнивая результаты более сложных моделей с самой наивной, мы можем делать выводы о том, насколько далеко они вообще продвинулись в качестве предсказаний. Иногда может случиться такое, что константная модель оказывается не сильно хуже по качеству. Это может говорить либо о том, что еще есть смысл продолжить поиски "той самой" модели, либо структура данных такова, что ничего лучше константоной в принципе не найти, тогда в использовании более сложных ресурсозатратных моделей нет особого смысла. Создавать константные модели можно либо вручную, либо воспользоваться готовым алгоритмом <a href='https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html'>DummyClassifier</a> из пакета `sklearn`. В нашем случае подойдет модель DummyClassifier со стратегией `most_frequent`. Рекомендую также прочитать  <a href='https://progler.ru/blog/ml-klassifikatory-pustyshki-s-ispolzovaniem-sklearn'>статью</a>.

## Общий вывод

Были проанализированы различные классификационные модели: Решающее дерево, Случайный лес, Логистическая регрессия.

Каждая модель была обучена для различных сочетаний гиперпараметров; выведены наилучшие варианты для каждой модели.

Наилучшая точность  была достигнута для модели Решающего дерева:

Accuracy на валидационной выборке: 0.7993779160186625

Accuracy на тестовой выборке: 0.7822706065318819

Максимального значения точности (0.7993779160186625) модель достигает при max_depth=7.

<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера v.1 <a class="tocSkip"> </h2>    
    
  
У меня сложилось хорошее впечатление о проекте. Молодец! Изучены все параметры, построено несколько моделей классификации и оценено их качество. Осмысленная аналитика и дельная модельная работа - многое удалось как надо)
    
Отмечу отдельные положительные моменты проекта🙂:
    
- тебе удалось добиться очень хорошего качества, поздравляю!  
- при обучении моделей использована валидационная выборка и подбор гиперпараметров.    
    
Есть несколько моментов всего, на которые стоит ещё раз взглянуть, я указал их в моих комментариях по ходу проекта - выборки и моделирование. Предлагаю тебе доработать проект по моим комментариям, чтобы довести его до совершенства.

    
Если будут вопросы, обращайся, с удовольствием на них отвечу.  
</div>

<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера v.2 <a class="tocSkip"> </h2>    
    
  
Есть несколько моментов всего, на которые стоит ещё раз взглянуть, я указал их в моих комментариях по ходу проекта - 
-  Некорректно как-либо использовать тестовую выборку в подборе гиперпараметров, а также обучать модель на полном датасете. Нам следует обучить модель на обучающей выборке и  сделать предсказание методом `predict` на тестовой выборке для ОДНОЙ лучшей модели - победителя с предыдущего шага и рассчитать итоговую метрику. Исправь, пожалуйста.

<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера v.3 <a class="tocSkip"> </h2>    
    
     
  
Теперь почти идеально, молодец! Принимаю твой проект)
    
Могу посоветовать посмотреть видеоразбор <a href="https://www.youtube.com/watch?v=YTLDATQLK6w">Как обучить модель по шагам в Data Science? Урок для начинающих на примере задачи регрессии</a>
    
- Рекомендую тебе ознакомиться со статьями по интерпретации предсказания моделей:  <a href="https://habr.com/ru/post/428213/">SHAP</a>,   <a href="https://towardsdatascience.com/using-shap-values-to-explain-how-your-machine-learning-model-works-732b3f40e137">+ еще ссылка</a>.           
    
    
В этом проекте все шаги отработаны. С интересом читал твои размышления, код и выводы. Цель нашего проекта достигнута. Рад был тебе помочь)
    

  
</div>